

# Propaganda, start the `spark` session

> For SQL users, Spark SQL provides state-of-the-art SQL performance and maintains compatibility with Shark/Hive. In particular, like Shark, Spark SQL supports all existing Hive data formats, user-defined functions (UDF), and the Hive metastore.

> For Spark users, Spark SQL becomes the narrow-waist for manipulating (semi-) structured data as well as ingesting data from sources that provide schema, such as JSON, Parquet, Hive, or EDWs. It truly unifies SQL and sophisticated analysis, allowing users to mix and match SQL and more imperative programming APIs for advanced analytics.

> For open source hackers, Spark SQL proposes a novel, elegant way of building query planners. It is incredibly easy to add new optimizations under this framework.

> Internally, a structured query is a Catalyst tree of (logical and physical) relational operators and expressions.




In [1]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

During the session, we will use classes and functions exported by `pyspark`


In [2]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

Start the `SparkSession`

In [3]:
conf = SparkConf().setAppName("Spark SQL Illustrations")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("Spark SQL")
    .getOrCreate()
)

US Baby Names 1880-2017
=======================


Description
: US baby names provided by the SSA. 

This dataset contains all names used
for at least 5 children of either sex during a year. 


The file is made of `1924665` lines and  4 columns.

```
|-- name: string (nullable = true)
    |-- n: integer (nullable = true)
    |-- sex: string (nullable = true)
    |-- year: integer (nullable = true)
```

Each row indicates for a given name, sex, and year the number of babies 
of the given sex who were given that name during the given year. Names 
with less than 5 occurrences during the year were note recorded. 

|    name|  n|sex|year|
|:--------|:---:|:---:|:----:|
|  Emilia|112|  F|1985|
|   Kelsi|112|  F|1985|
|  Margot|112|  F|1985|
|  Mariam|112|  F|1985|
|Scarlett|112|  F|1985|

First, we download the data if it's not there yet

In [4]:
import requests, zipfile, io
from pathlib import Path

path = Path('babynames_short.csv')
if not path.exists():
    url = "https://stephanegaiffas.github.io/big_data_course/data/babynames_short.csv.zip"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path='./')

Load `babynames` from a `csv` file

In [5]:
df_sp = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("babynames_short.csv")

df_sp.printSchema()

root
 |-- name: string (nullable = true)
 |-- n: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- year: double (nullable = true)




Ensure that the dataframe has the following schema:

    root
        |-- name: string (nullable = true)
        |-- n: integer (nullable = true)
        |-- sex: string (nullable = true)
        |-- year: integer (nullable = true)




In [7]:
df_sp.show()

+----------+---+---+------+
|      name|  n|sex|  year|
+----------+---+---+------+
|    Emilia|112|  F|1985.0|
|     Kelsi|112|  F|1985.0|
|    Margot|112|  F|1985.0|
|    Mariam|112|  F|1985.0|
|  Scarlett|112|  F|1985.0|
|      Aida|111|  F|1985.0|
|    Ashlei|111|  F|1985.0|
|     Greta|111|  F|1985.0|
|    Jaimee|111|  F|1985.0|
|     Lorna|111|  F|1985.0|
|   Rosario|111|  F|1985.0|
|     Sandi|111|  F|1985.0|
|   Sharina|111|  F|1985.0|
|    Tashia|111|  F|1985.0|
|     Adina|110|  F|1985.0|
|    Ahsley|110|  F|1985.0|
|Alessandra|110|  F|1985.0|
|    Amalia|110|  F|1985.0|
|    Chelsi|110|  F|1985.0|
|    Darcie|110|  F|1985.0|
+----------+---+---+------+
only showing top 20 rows



SQL versus spark-Dataframe API
=================================

>  Dataset API vs SQL

> Spark SQL supports two "modes" to write structured queries: Dataset API and SQL. SQL Mode is used to express structured queries using SQL statements using SparkSession.sql operator, expr standard function and spark-sql command-line tool.

> Some structured queries can be expressed much easier using Dataset API, but there are some that are only possible in SQL. In other words, you may find mixing Dataset API and SQL modes challenging yet rewarding.

> What is important, and one of the reasons why Spark SQL has been so successful, is that there is no performance difference between the modes. Whatever mode you use to write your structured queries, they all end up as a tree of Catalyst relational data structures. And, yes, you could consider writing structured queries using Catalyst directly, but that could quickly become unwieldy for maintenance (i.e. finding Spark SQL developers who could be comfortable with it as well as being fairly low-level and therefore possibly too dependent on a specific Spark SQL version).

Warmup:  compute the 10 most popular names given to babies in year 2000.
======================================================================

## Using `spark.sql()`

In order to use mode `sql`, create a temporary view from the `DataFrame`.

1. What are temporary views made of?
1. Are there other kind of views in spark's world?

In [6]:
#1. Les "temporary views" dans Spark SQL sont limitées à la session et disparaissent si la session qui les a créées se termine.
#2. Il y a "TempView" et "GlobalTempView"

[]

In [27]:
# TODO: 

df_sp.createOrReplaceTempView("new_view")


## A query is a plain SQL query embodied in a string.



In [181]:
query = "SELECT sex, year FROM new_view WHERE name='Emilia'"
emilia_df = spark.sql(query)
emilia_df.limit(10).show()

+---+------+
|sex|  year|
+---+------+
|  F|1985.0|
|  F|1986.0|
|  F|1987.0|
|  F|1988.0|
|  F|1989.0|
|  F|1990.0|
|  F|1991.0|
|  F|1992.0|
|  F|1993.0|
|  F|1994.0|
+---+------+




> This phrasing is not consistent with the DRY principle. Fix this using formatted strings.

## Using the dataframe/dataset API

This can also be done using Spark SQL API.

### Pedestrian approach

1. First select `10` most popular names for girls in year `2000`, define `spark` dataframe
`top10_2000_f`.
1. Does the definition of `top10_2000_f` involve _transformations_, _actions_ or both?
1. What is the type of the result returned by `top10_2000_f.take(2)`? the type of elements of the result?



In [269]:
query = """
SELECT name, sum(n) AS count 
FROM new_view 
WHERE year=2000.0 and sex='F' 
GROUP BY name 
ORDER BY count DESC 
LIMIT 10
"""
top10_2000_f = spark.sql(query)
top10_2000_f.show()

+---------+-----+
|     name|count|
+---------+-----+
|    Emily|25953|
|   Hannah|23080|
|  Madison|19967|
|   Ashley|17997|
|    Sarah|17697|
|   Alexis|17629|
| Samantha|17266|
|  Jessica|15709|
|Elizabeth|15094|
|   Taylor|15078|
+---------+-----+



In [103]:
top10_2000_f.take(2)
# le type du résultat est une liste

[Row(name='Kiana', count=1), Row(name='Heaven', count=1)]


1. Do the same thing for boys.



In [270]:
query = """
SELECT name, sum(n) AS count 
FROM new_view 
WHERE year=2000.0 and sex='M' 
GROUP BY name 
ORDER BY count DESC 
LIMIT 10
"""
top10_2000_m = spark.sql(query)
top10_2000_m.show()

+-----------+-----+
|       name|count|
+-----------+-----+
|      Jacob|34471|
|    Michael|32035|
|    Matthew|28572|
|     Joshua|27538|
|Christopher|24931|
|   Nicholas|24652|
|     Andrew|23639|
|     Joseph|22825|
|     Daniel|22312|
|      Tyler|21503|
+-----------+-----+




1. Compute the _union_ of the two spark dataframes. Store the result in
dataframe `top10_2000`



In [271]:
top10_2000 = top10_2000_m.unionAll(top10_2000_f)
top10_2000.show()

+-----------+-----+
|       name|count|
+-----------+-----+
|      Jacob|34471|
|    Michael|32035|
|    Matthew|28572|
|     Joshua|27538|
|Christopher|24931|
|   Nicholas|24652|
|     Andrew|23639|
|     Joseph|22825|
|     Daniel|22312|
|      Tyler|21503|
|      Emily|25953|
|     Hannah|23080|
|    Madison|19967|
|     Ashley|17997|
|      Sarah|17697|
|     Alexis|17629|
|   Samantha|17266|
|    Jessica|15709|
|  Elizabeth|15094|
|     Taylor|15078|
+-----------+-----+




### Do it again, complying  with DRY principle



In [280]:
top10_2000_f = df_sp\
                .where("year==2000.0")\
                .where("sex='M'")\
                .groupBy("name")\
                .agg(fn.sum(col('n')).alias('sum'))\
                .orderBy(col("sum").desc())\
                .limit(10)\
                .show()

+-----------+-----+
|       name|  sum|
+-----------+-----+
|      Jacob|34471|
|    Michael|32035|
|    Matthew|28572|
|     Joshua|27538|
|Christopher|24931|
|   Nicholas|24652|
|     Andrew|23639|
|     Joseph|22825|
|     Daniel|22312|
|      Tyler|21503|
+-----------+-----+



In [192]:
top10_2000_m = df_sp\
                .where("year==2000.0")\
                .where("sex='M'")\
                .groupBy("name")\
                .agg(fn.sum(col('n')).alias('sum'))\
                .orderBy(col("sum").desc())\
                .limit(10)\
                .show()

+-----------+-----+
|       name|  sum|
+-----------+-----+
|      Jacob|34471|
|    Michael|32035|
|    Matthew|28572|
|     Joshua|27538|
|Christopher|24931|
|   Nicholas|24652|
|     Andrew|23639|
|     Joseph|22825|
|     Daniel|22312|
|      Tyler|21503|
+-----------+-----+





Name portfolio through ages
===========================

1. Compute for each year and sex the number of distinct names given that year.



In [157]:
nb_names_year_sex = df_sp\
                    .groupBy("year", "sex")\
                    .agg(fn.countDistinct("name")\
                    .alias('count'))\
                    .orderBy("year")
nb_names_year_sex.show()

+------+---+-----+
|  year|sex|count|
+------+---+-----+
|1880.0|  M| 1058|
|1880.0|  F|  942|
|1881.0|  M|  997|
|1881.0|  F|  938|
|1882.0|  F| 1028|
|1882.0|  M| 1099|
|1883.0|  F| 1054|
|1883.0|  M| 1030|
|1884.0|  F| 1172|
|1884.0|  M| 1125|
|1885.0|  F| 1197|
|1885.0|  M| 1097|
|1886.0|  M| 1110|
|1886.0|  F| 1282|
|1887.0|  F| 1306|
|1887.0|  M| 1067|
|1888.0|  M| 1177|
|1888.0|  F| 1474|
|1889.0|  M| 1111|
|1889.0|  F| 1479|
+------+---+-----+
only showing top 20 rows




1. Plot the evolution of the number of distinct names as a function of `year`.
Use some aesthetics to distinguish sexes.




In [194]:
# TODO:
import plotly.graph_objects as go

nb_names = nb_names_year_sex\
            .toPandas()\
            .pivot(index='year',columns='sex',values='count')\
            .rename_axis([None],axis=1)\
            .reset_index()

fig = go.Figure(layout={"title":"The number of distinct names of year",
                        "xaxis_title": "year",
                        "yaxis_title": "The number of distinct names"})

fig.add_trace(go.Scatter(x=nb_names['year'], 
                         y=nb_names['F'],
                         mode='lines',
                         name='Female'))

fig.add_trace(go.Scatter(x=nb_names['year'], 
                         y=nb_names['M'],
                         mode='lines',
                         name='Male'))
fig.show()




Assessing popularity through time
=================================

1. For each year and sex, compute the total number of births
1. Plot the evolution of the sex ratio over time
1. For each year, sex, and name compute the percentage of newborns
given that name for that given year.


> Use `Window` functions.



In [234]:
# TODO:
births = df_sp\
        .groupBy('year','sex')\
        .agg(fn.sum('n').alias('number'))

window = Window.partitionBy('year')
ratio = fn.col('number')/fn.sum('number').over(window)
births = births.withColumn('ratio',fn.round(ratio*100, 2))
births.show(10)

+------+---+-------+-----+
|  year|sex| number|ratio|
+------+---+-------+-----+
|1988.0|  M|1913203| 51.8|
|1988.0|  F|1780268| 48.2|
|1951.0|  M|1882982|51.11|
|1951.0|  F|1800880|48.89|
|1976.0|  M|1569843|51.73|
|1976.0|  F|1465106|48.27|
|1940.0|  F|1143393|49.66|
|1940.0|  M|1158985|50.34|
|1928.0|  F|1153221|51.01|
|1928.0|  M|1107618|48.99|
+------+---+-------+-----+
only showing top 10 rows



In [236]:
# %%
# TODO: plot sex ratio
# %%
plot_births = births\
            .toPandas()\
            .pivot(index='year',columns='sex',values='ratio')\
            .rename_axis([None],axis=1)\
            .reset_index()

fig = go.Figure(layout={"title":"The sex ratio per year",
                        "xaxis_title": "year",
                        "yaxis_title": "The sex ratio"})

fig.add_trace(go.Scatter(x=plot_births['year'], 
                         y=plot_births['F'],
                         mode='lines',
                         name='Female'))

fig.add_trace(go.Scatter(x=plot_births['year'], 
                         y=plot_births['M'],
                         mode='lines',
                         name='Male'))
fig.show()

In [250]:
# For each year, sex, and name compute the percentage of newborns given that name for that given year.
name_per_newborns = df_sp\
                    .groupBy('year','sex','name')\
                    .agg(fn.sum('n').alias('number'))

window = Window.partitionBy('year')
percen = fn.col('number')/fn.sum('number').over(window)
name_per_newborns = name_per_newborns.withColumn('percentage',fn.round(percen*100,6))
name_per_newborns.show(10)


+------+---+---------+------+----------+
|  year|sex|     name|number|percentage|
+------+---+---------+------+----------+
|1988.0|  F| Tanielle|    11|   2.98E-4|
|1988.0|  F|   Kortne|     9|   2.44E-4|
|1988.0|  F|    Daija|     8|   2.17E-4|
|1988.0|  F|    Jaide|     8|   2.17E-4|
|1988.0|  F|Stefannie|     8|   2.17E-4|
|1988.0|  F|Catharina|     7|    1.9E-4|
|1988.0|  F|   Shieka|     7|    1.9E-4|
|1988.0|  F|  Shaneal|     6|   1.62E-4|
|1988.0|  F|    Bhumi|     5|   1.35E-4|
|1988.0|  F|  Kejuana|     5|   1.35E-4|
+------+---+---------+------+----------+
only showing top 10 rows




1. Compute for each year, sex and name  the `row_number`, `rank`, and `dense_rank`
of the name within that year and sex category, when names are sorted by increasing popularity.



In [260]:
# TODO:
name_number = df_sp\
              .groupBy('year','sex','name')\
              .agg(fn.sum('n').alias('number'))

window = Window.partitionBy('year', 'sex')\
        .orderBy(name_number.number.desc())

rank = fn.rank().over(window)
dense_rank = fn.dense_rank().over(window)
row_number = fn.row_number().over(window)

name_number = name_number\
        .withColumn('rank', rank)\
        .withColumn('dense_rank', dense_rank)\
        .withColumn('row_number', row_number)\
        .orderBy(fn.col('number').desc())\
        .show()



+------+---+-------+------+----+----------+----------+
|  year|sex|   name|number|rank|dense_rank|row_number|
+------+---+-------+------+----+----------+----------+
|1947.0|  F|  Linda| 99686|   1|         1|         1|
|1948.0|  F|  Linda| 96209|   1|         1|         1|
|1947.0|  M|  James| 94756|   1|         1|         1|
|1957.0|  M|Michael| 92695|   1|         1|         1|
|1947.0|  M| Robert| 91642|   2|         2|         2|
|1949.0|  F|  Linda| 91016|   1|         1|         1|
|1956.0|  M|Michael| 90620|   1|         1|         1|
|1958.0|  M|Michael| 90520|   1|         1|         1|
|1948.0|  M|  James| 88588|   1|         1|         1|
|1954.0|  M|Michael| 88514|   1|         1|         1|
|1955.0|  M|Michael| 88335|   1|         1|         1|
|1947.0|  M|   John| 88318|   3|         3|         3|
|1946.0|  M|  James| 87431|   1|         1|         1|
|1951.0|  M|  James| 87272|   1|         1|         1|
|1952.0|  M|  James| 87107|   1|         1|         1|
|1961.0|  




Evolution of top popular names through the century
==================================================


1. For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.



In [ ]:
# TODO:


1. Use `explain()` to determine the joining strategy used by spark.


Plot  the popularity of each of the top ten achievers from year 2000 with respect to time
==================================================================================




In [ ]:
# %%
# TODO:
# %%



Plot the total popularity of the top ten achievers from year 2000 with respect to time
==================================================================================




In [ ]:
# %%
# TODO:
# %%



Plot lorenz curves
=====================

Every year, the name counts define a discrete probability distribution.
This distribution, just as income or wealth distribution,
is (usually) far from being uniform. We want to assess how uneven it is.
We use the tools developed in econometrics.

Without loss of generality, that we handle a distribution over $1, \ldots, n$
where $n$ is the number of distinct names given during a year.
We assume that frequencies $p_1, p_2, \ldots, p_n$ are given in ascending order.

The Lorenz function maps $[0, 1] \to [0, 1]$.
$$L(x) = \sum_{i=1}^{\lfloor nx \rfloor} p_i$$.

1. Design a query that adds a column "lorenz" to the dataframe , and for each
row computes the value of the Lorenz function defined by `year`  and `sex`.




In [ ]:
# TODO:



1. Design a function that takes as input a `year` and plots the Lorenz curve
for that year for both sexes.



In [ ]:
# %%
# TODO:
# %%


Gini index
==========

The [Gini index](https://en.wikipedia.org/wiki/Gini_coefficient) is twice the surface of the area comprised between curves $y=x$
and $y=L(x)$.

Choose a formula that allows you to compute it efficiently.

$$G={\frac {2\sum _{i=1}^{n}iy_{i}}{n\sum _{i=1}^{n}y_{i}}}-{\frac {n+1}{n}}.$$

1. Design a query that computes the Gini index of the `babynames` distribution
for every `year` and `sex`.

1. Plot Gini index over time




In [ ]:
# TODO:

In [ ]:
# %%
# TODO:
# %%




Close the door, leave work area clean
=====================================


In [ ]:
spark.stop()